<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align="left">

# Static Secrets (v1)
---
Key/Value Secrets engine is used to store arbitrary secrets.  It can be a generic Key-Value store that stores one value for a key. Or Versioning can be enabled and a number of versions for each key will be stored.

**Prerequisites:**
* Vault binary

EXECUTE the following command in SEPARATE window
```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 evault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```
 
### Setup
**Note**:  
* Run > Render All Markdown Cells
* Edit > Clear all outputs

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status; vault version

---
### Demo: Your First Secret
Now that the Vault server is up and running, let's get read and write your first secret.  One of the core features of Vault is the ability to read and write arbitrary secrets securely.  Secrets are written to Vaul are encrypted and then written to backend storage.

Let's write a secret to Key/Value secret engine.  
   
**Note**:  
* Run > Render All Markdown Cells
* Edit > Clear all outputs

In [ ]:
vault secrets enable -version=1 -path=secret1 kv

### Write key/value secrets
This writes multiple data, `username` and `password`, to the path `secret1/training`.  The `secret1/` prefix is where the secret engine is.  Vault offers the concepth of paths which means that engines, policies, and data can be applied to different paths to control access.

In [ ]:
vault kv put secret1/training username="student01" password="r31nsta11"

In [ ]:
vault kv put secret1/web01 ip="192.168.1.191" username="arthur" password="passw0rd123"
vault kv put secret1/web02 ip="192.168.1.192" username="raymond" password="passw0rd123"

In [ ]:
curl -X PUT -H "X-Vault-Token: $(vault print token)" \ 
-d '{"ip": "192.168.1.192", "password":"r31nsta11","username":"student01"}' http://127.0.0.1:8200/v1/secret1/training

### Read the secrets

In [ ]:
vault kv get secret1/training

In [ ]:
curl -H "X-Vault-Token: $(vault print token)"  http://127.0.0.1:8200/v1/secret1/training |jq .

In [ ]:
vault kv get -format=json secret1/training | jq -r .data

In [ ]:
vault kv get -field=username secret1/training

### Use response wrapping  
When requested, Vault can insert the secret into `cubbyhole` of a single-use token, returning that single-use token instead.  
Use case: provide cover for the secret information, detect interception or tampering and limit lifetime of secret exposure.

In [ ]:
vault kv get -output-curl-string -wrap-ttl=3m secret1/training

&nbsp;

In [ ]:
vault unwrap s.TVe2ebDexeTcHuefRMKKh8L8

**Perform a lookup on response-wrapping token, this immediately tells you if the token has already been unwrapped or is expired**

In [ ]:
curl -X PUT -H "X-Vault-Token: $(vault print token)" -d '{"token":"s.2DuCg2mFzef2cl9ifacRxGiq"}' http://127.0.0.1:8200/v1/sys/wrapping/lookup  | jq .

### List all the secret keys stored in the K/V Secret backend

In [ ]:
vault kv list secret1

### Delete Secrets

In [ ]:
VAULT_TOKEN=root vault kv delete secret1/training

## API

In [ ]:
#-output-curl-string
vault kv put -output-curl-string secret1/apikey/google apikey="xvafdoygad1158"

**execute the cURL command**

In [ ]:
curl -X PUT -H "X-Vault-Token: $(vault print token)" \
   -d '{"apikey":"xvafdoygad1157"}' \
   http://127.0.0.1:8200/v1/secret1/apikey/google

**Read the secret **

In [ ]:
curl -s -H "X-Vault-Token: $(vault print token)" http://127.0.0.1:8200/v1/secret1/apikey/google   | jq .data.apikey

In [ ]:
vault kv get secret1/apikey/google

**Delete the secret**

In [ ]:
curl -s -X DELETE -H "X-Vault-Token: $(vault print token)" http://127.0.0.1:8200/v1/secret1/apikey/google 

---
## Upgrading from Version 1 
An existing version 1 KeyValue can be upgraded to a version 2 KeyValue store

In [ ]:
vault kv enable-versioning secret1/

---
### Via UI
Open a web browser and visit http://127.0.0.1:8200/ui/vault Use TOKEN and Sign in


**TODO:**. 
- delete vs destroy with version

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">